<a href="https://colab.research.google.com/github/dhyeybhatpuria/Crux_Alliance/blob/master/Ex2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Exercise 2**

Since Visualisation of layers is done using GEEMAP package. Below line will install geemap to colab

In [43]:
!pip install geemap

Run the below line and Please provide permission to access colab and also GEE credentials with all access. 

In GEE notebook authenticator, select default cloud project, Set data access to read-only scopes. 
Allow:
*   View your Google Earth Engine data
*   View your data in Google Cloud Storage.
Copy the code generated in box within below line "Verification Code:"

Code in notebook requires these permissions to run using free quota of GEE and it will not copy, modify any existing datasets in your GEE assets or Google Cloud Platform (GCP). 

In [44]:
# Installs geemap package
import google.colab
from google.colab import auth
auth.authenticate_user()
import geemap.eefolium as geemap
# Authenticates and initializes Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=phJBkjdx5Xo12fPPvQvgSuQ1oqLq8XwMEyeoSuAu-R4&tc=d_Ejcmk2kDt4HWA6onNNiWjCdX5AgvZuy0qPQovFMkg&cc=iMVBeXONgEtB1LnSG4qrP5AlbJk4ljLOh6m5WQZXKTk

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjwFu4mYKbFvm-xsaR40mVNg04uCauDrSM8JyvJrRfKxe-AcToM0rw

Successfully saved authorization token.


Calculation of Indices \
Formula of NDVI is $(NIR-RED)/(NIR+RED)$ \
Formula of NDWI is $(GREEN-NIR)/(GREEN+NIR)$

In [56]:
# Create functions for calculating NDVI and NDWI and append them to the image as a band
def calc_indices(image):
  ndvi = (image.normalizedDifference(['B5', 'B4'])).rename("NDVI") # B5 is NIR ,B4 is Red
  ndwi = (image.normalizedDifference(['B3', 'B5'])).rename("NDWI") # B5 is NIR ,B3 is Green
  return image.addBands(ndvi).addBands(ndwi)

In [57]:
# Load Landsat 
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
areaofint = ee.FeatureCollection("users/dhyeybhatpuria/India_Boundary")
Landsat_indices = landsat \
  .filterDate('2021-01-01', '2021-12-31') \
  .map(calc_indices) \
  .median() \
  .clip(areaofint)

In [58]:
# Visualisation parameter
import geemap.colormaps as cm
vis_parameters = {'bands': ['B4', 'B3', 'B2'], 'min':0, 'max': 0.3};
# colorpalette1 =  ['FFFFFF','CC9966','CC9900', '996600', '33CC00', '009900','006600','000000']
colorpalette =  ['green', 'yellow','red'] # Standard palette
# geemap based palette
palette_ndvi = cm.palettes.ndvi
ndvi_parameter = {'min':-0.5, 'max': 1, 'palette':palette_ndvi}
palette_ndwi = cm.palettes.ndwi
ndwi_parameter = {'min':-0.5, 'max': 1, 'palette':palette_ndwi}

In [59]:
Map = geemap.Map(center=[23,78], zoom= 5)
Map.addLayer(Landsat_indices.select("NDWI"), ndwi_parameter, 'Median NDWI')
Map.addLayer(Landsat_indices.select("NDVI"), ndvi_parameter, 'Median NDVI')
# Map.add_colorbar(ndwi_parameter, label= "NDWI", layer_name = "Median NDWI")
# Map.add_colorbar(ndvi_parameter,label= "Index", layer_name="Median NDVI")
Map.addLayerControl()
Map
# Toggle layers from layers tab in right side of the map area

**Exercise 3**

In [69]:
modis_lulc = ee.ImageCollection('MODIS/006/MCD12Q1')
# filter for date, select only IGBP LULC
lc = modis_lulc \
  .filterDate('2020-01-01', '2020-12-31') \
  .select('LC_Type1') \
  .first()


agricultural_mask = lc.eq(12) \
  .selfMask()

forest_lc = (lc.lte(10)).multiply(1)
agro_lc = (lc.eq(12).Or(lc.eq(14))).multiply(2)
urban_lc = (lc.eq(13)).multiply(3)
water_lc = (lc.eq(17)).multiply(4)
other_lc = (((lc.eq(15)).Or(lc.eq(16))).Or(lc.eq(11))).multiply(5)
classified = forest_lc.add(agro_lc).add(urban_lc).add(water_lc).add(other_lc)

agricultural_mask = classified.eq(2).selfMask()

In [71]:
ndvi_layer = Landsat_indices.select("NDVI")
agro_ndvi = (ee.Image(ndvi_layer) \
             .mask(agricultural_mask)) \
             .clip(areaofint)

On running below line please check in right tab to toggle LULC and Masked NDVI

In [76]:
igbpLandCoverVis = {'min': 1.0, 'max': 5.0, 'palette': ['green','yellow','brown','blue','gray']}

Map = geemap.Map(center=[23,78], zoom= 5)
Map.addLayer(classified.clip(areaofint), igbpLandCoverVis, 'Classified LULC')
# Green = forest, Yellow = Agriculture, Brown = Urban, Blue = water and Gray = Others
Map.addLayer(agro_ndvi.clip(areaofint), ndvi_parameter, 'NDVI-Agriculture land only')

# Map.add_colorbar(ndwi_parameter, label= "NDWI", layer_name = "Median NDWI")
# Map.add_colorbar(ndvi_parameter,label= "Index", layer_name="Median NDVI")
Map.addLayerControl()
Map

Map(center=[23, 78], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

Exercise 4
Lastly provide statistics from the four classes derived from the MODIS land cover map conducted in exercise 3.

In [78]:
# zonal stats failed
# stats = geemap.image_stats_by_zone(ndvi_layer, classified, reducer='MEAN')
# stats

# Histogram of pixel count for each land cover type
import geemap.chart as chart
my_sample = classified.sample(areaofint, 5000)
property = 'LC_Type1'
options = {
    "title": 'July Precipitation Distribution for NW USA',
    "xlabel": 'Precipitation (mm)',
    "ylabel": 'Pixel count',
    "colors": ['#1d6b99'],
}
chart.feature_histogram(my_sample, property, **options)